In [6]:
import pandas as pd
import numpy as np
import gensim 
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv('supervised_sample_datasets/lexicon_step1.csv')
df

,Unnamed: 0,created_at,user_id,username,tweet,place,near,clean_tweet,tokens,lemma,lemma_sentence,pos_tag,lemma_sentence(with POS),sentiword_analysis,vader_score,textblob_polarity,senti_textblob,senti_wordnet,senti_vader
0,0,2021-01-06,293175196,sjtafalla,UK Parliament: 2nd Covid Vaccine should be 21 ...,NaN,London,parliament covid vaccine days not weeks challe...,"['parliament', 'covid', 'vaccine', 'days', 'no...","['parliament', 'covid', 'vaccine', 'day', 'not...",parliament covid vaccine day not week challeng...,"[('parliament', 'n'), ('covid', 'n'), ('vaccin...",parliament covid vaccine day not week challe...,-0.625,-0.0572,0.000000,0,-1,-1
1,1,2021-01-06,1591779799,ellieelif,First dose of vaccination 💉 5/1/2021..to comba...,NaN,London,first dose vaccination syringe combating covid,"['first', 'dose', 'vaccination', 'syringe', 'c...","['first', 'dose', 'vaccination', 'syringe', 'c...",first dose vaccination syringe combating covid,"[('first', 'r'), ('dose', 'a'), ('vaccination'...",first dose vaccination syringe combat covid,-0.125,-0.3400,0.250000,1,-1,-1
2,2,2021-01-06,215143656,danananarama,"Time to forget about #COVID, #Brexit and #Trum...",NaN,London,time forget covid brexit trump sleep listening...,"['time', 'forget', 'covid', 'brexit', 'trump',...","['time', 'forget', 'covid', 'brexit', 'trump',...",time forget covid brexit trump sleep listening...,"[('time', 'n'), ('forget', 'v'), ('covid', 'a'...",time forget covid brexit trump sleep listeni...,0.250,-0.2263,0.000000,0,1,-1
3,3,2021-01-06,336462129,veronica_foote_,@doctor_oxford Rachel you absolutely nailed it...,NaN,London,rachel absolutely nailed tonight throughout pr...,"['rachel', 'absolutely', 'nailed', 'tonight', ...","['rachel', 'absolutely', 'nailed', 'tonight', ...",rachel absolutely nailed tonight throughout pr...,"[('rachel', 'n'), ('absolutely', 'r'), ('naile...",rachel absolutely nail tonight throughout pr...,1.375,0.6124,0.300000,1,1,1
4,4,2021-01-06,1063705581133934593,5herii,My kids can never say they don’t wanna do thei...,NaN,London,kids never say not wana homework got sparkles ...,"['kids', 'never', 'say', 'not', 'wana', 'homew...","['kid', 'never', 'say', 'not', 'wana', 'homewo...",kid never say not wana homework got sparkle co...,"[('kids', 'n'), ('never', 'r'), ('say', 'v'), ...",kid never say not wana homework get sparkle ...,-0.750,0.6573,0.025000,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29929,29929,2021-01-30,1200125350694576128,MayorJesse,Health Rover will be offering rapid COVID-19 a...,"{'type': 'Feature', 'bbox': [-72.4243, 40.8667...","Southampton, NY",health rover offering rapid covid antigen test...,"['health', 'rover', 'offering', 'rapid', 'covi...","['health', 'rover', 'offering', 'rapid', 'covi...",health rover offering rapid covid antigen test...,"[('health', 'n'), ('rover', 'n'), ('offering',...",health rover offering rapid covid antigen te...,0.750,0.0000,0.000000,0,1,0
29930,29930,2021-01-27,803004340608843776,thatmikeny,#Americaorbust FOX complicit in so Manet death...,"{'type': 'Feature', 'bbox': [-72.4243, 40.8667...","Southampton, NY",americaorbust fox complicit manet deaths due c...,"['americaorbust', 'fox', 'complicit', 'manet',...","['americaorbust', 'fox', 'complicit', 'manet',...",americaorbust fox complicit manet death due co...,"[('americaorbust', 'a'), ('fox', 'n'), ('compl...",americaorbust fox complicit manet death due ...,-0.625,-0.8126,-0.125000,-1,-1,-1
29931,29931,2021-01-21,408755761,ConstanceHunter,Here is @POTUS plan to defeat #COVID19. 👇🏻\nSt...,"{'type': 'Feature', 'bbox': [-72.4243, 40.8667...","Southampton, NY",plan defeat covid backhand index pointing down...,"['plan', 'defeat', 'covid', 'backhand', 'index...","['plan', 'defeat', 'covid', 'backhand', 'index...",plan defeat covid backhand index pointing down...,"[('plan', 'n'), ('defeat', 'v'), ('covid', 'a'...",plan defeat covid backhand index point down ...,-0.125,-0.6369,0.122222,1,-1,-1
29932,29932,2021-

In [8]:
#sample data
positive_df = df[df["senti_vader"] == 1]
positive_df = positive_df[:1000] #1000 positive sentiment
neutral_df = df[df["senti_vader"] == 0]
neutral_df = neutral_df[:1000] #1000 neutral sentiment
negative_df = df[df["senti_vader"] == -1]
negative_df = negative_df[:1000] #1000 neutral sentiment

In [9]:
# df=[positive_df, neutral_df, negative_df]
# df=pd.concat(df)
# df=df.reset_index(drop=True)
# df.to_csv('supervised_sample_datasets/sample_data.csv', index=False, encoding='utf_8_sig')

In [10]:
#after manual check
df = pd.read_csv('supervised_sample_datasets/sample_data.csv')

In [11]:
#after manual check
negative_num=len(df[df['senti_vader'] < 0])
print("negative:", negative_num)
neutral_num=len(df[df['senti_vader'] == 0])
print("neutral", neutral_num)
positive_num=len(df[df['senti_vader'] > 0])
print("positive", positive_num) 

negative: 996
neutral 1009
positive 995


In [12]:
# df=df.drop('Unnamed: 0', axis=1)

In [13]:
#BoW
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from nltk.tokenize import word_tokenize

def bag_of_words(df):
#     bow_vectorizer = CountVectorizer(max_df=0.90, min_df=0.2, stop_words=None, tokenizer=word_tokenize) 
    bow_vectorizer = CountVectorizer() 
    bow = bow_vectorizer.fit_transform(df['lemma_sentence(with POS)']) 
    #print(bow_vectorizer.get_feature_names())
    #print(bow_vectorizer.vocabulary_)
    return bow

df_bow=bag_of_words(df)
df_bow.shape

(3000, 7363)

In [14]:
#TF-IDF
def tf_idf(df):
#     tf_idf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.2, stop_words=None, tokenizer=word_tokenize, norm='l2') 
    tf_idf_vectorizer = TfidfVectorizer(norm='l2') #extract features
    tfidf = tf_idf_vectorizer.fit_transform(df['lemma_sentence(with POS)']) #vectors
    return tfidf
df_tfidf=tf_idf(df)
df_tfidf.shape

(3000, 7363)

In [15]:
#Word2vec
#reference：https://www.pythonf.cn/read/93491
#https://github.com/Shwetago/Sentiment_Analysis/blob/master/Twitter_Sentiment_Analysis.ipynb
from nltk.tokenize import word_tokenize
Tokenize_tweet = df['lemma_sentence(with POS)'].apply(word_tokenize)
print(Tokenize_tweet)

Model_W2V = gensim.models.Word2Vec(Tokenize_tweet, size=200, #features
                                   window=5, 
                                   min_count=1, 
                                   sg=1,  #skip-gram model
                                   hs=0,
                                   negative=10, 
                                   workers=2, 
                                   seed=34 ) 

0       [rachel, absolutely, nail, tonight, throughout...
1       [kid, never, say, not, wana, homework, get, sp...
2       [not, mess, covid, wear, mask, time, london, u...
3       [problem, think, airfield, cost, remain, open,...
4       [remain, astonished, stock, market, not, much,...
                              ...                        
2995    [woke, today, ping, bank, notify, hospital, de...
2996    [football, ban, till, covid, control, think, e...
2997    [useless, company, affect, financially, cause,...
2998    [listen, covid, death, high, people, learn, di...
2999    [friend, live, dubai, call, yesterday, say, si...
Name: lemma_sentence(with POS), Length: 3000, dtype: object


In [16]:
#Each word can get its own vector. The representation of a tweets can the vector sum of each word divided by the total number(average) 
#or just the sum of each word vector
def word2vec_tweet(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V[word].reshape((1, size))
        vector_cnt += 1
    return vector/vector_cnt  #average for tweets

def word2vec_tweet_2(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V[word].reshape((1, size))
    return vector  #sum of tweets

tweet_arr=np.zeros((len(Tokenize_tweet), 200))

for i in range (len(Tokenize_tweet)):
    tweet_arr[i,:] = word2vec_tweet(Tokenize_tweet[i], 200)
tweet_vec_df = pd.DataFrame(tweet_arr)
tweet_vec_df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.046028,0.013125,-0.215196,0.054006,-0.045560,0.046852,0.008420,0.032719,-0.166786,-0.141116,...,0.027627,0.169176,-0.012951,-0.070157,0.261464,0.042586,0.028671,0.163418,-0.131032,0.012715
1,0.057419,-0.007375,-0.220125,0.045612,-0.039521,0.026838,-0.015003,0.033050,-0.163985,-0.140885,...,0.043693,0.178724,-0.029015,-0.078524,0.260154,0.032928,0.034161,0.176410,-0.137545,0.034150
2,0.062423,-0.006053,-0.242087,0.033288,-0.054310,0.037971,-0.011595,0.040159,-0.163437,-0.135993,...,0.030736,0.196103,-0.033956,-0.082589,0.266228,0.046577,0.028694,0.178976,-0.131259,0.034813
3,0.052720,-0.001593,-0.214062,0.045373,-0.039554,0.033358,-0.009572,0.030766,-0.158312,-0.134943,...,0.037588,0.169993,-0.022311,-0.074826,0.250775,0.032766,0.032073,0.168943,-0.132774,0.027918
4,0.061093,-0.013015,-0.227416,0.034307,-0.040996,0.023911,-0.020089,0.036286,-0.158784,-0.134447,...,0.043168,0.185263,-0.036659,-0.080655,0.258209,0.030195,0.034023,0.175576,-0.133142,0.035248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.061576,-0.015965,-0.209558,0.039427,-0.036717,0.015143,-0.021524,0.034016,-0.156649,-0.125417,...,0.046313,0.168563,-0.029884,-0.079577,0.242419,0.033425,0.036464,0.166718,-0.132280,0.039234
2996,0.057106,-0.005136,-0.229884,0.045231,-0.041574,0.029140,-0.013771,0.032535,-0.169214,-0.144692,...,0.042866,0.183323,-0.028413,-0.080946,0.268426,0.035118,0.033613,0.180557,-0.144866,0.032142
2997,0.069067,-0.019276,-0.233933,0.041691,-0.038708,0.017015,-0.027743,0.033497,-0.172945,-0.144220,...,0.055926,0.192329,-0.037111,-0.088606,0.270958,0.031635,0.039046,0.194003,-0.146554,0.047267
2998,0.059230,-0.010724,-0.206712,0.040984,-0.036715,0.019315,-0.018032,0.033516,-0.154064,-0.129493,...,0.044305,0.165126,-0.029433,-0.075435,0.244689,0.036557,0.034476,0.163093,-0.133008,0.033050


In [44]:
#BoW for three classification models
#split the train and test datasets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2)

In [45]:
#parameters in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] #tree number
max_features = ['auto', 'sqrt','log2']
max_depth = [10,20,30,40]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5, 10]

# Create the param grid
param_grid_forest = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid_forest)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 5, 10]}


In [46]:
#parameters in MNB
param_grid_nb = {'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0],
                'fit_prior':[True, False]}
print(param_grid_nb)

{'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0], 'fit_prior': [True, False]}


In [47]:
#parameters in SVC
# c_list=list(range(1,51))
param_grid_svc = {'C': [1, 10, 100, 1000],
                  'kernel': ['linear','poly','rbf','sigmoid'],
                  'degree': [1,2,3,4]}
print(param_grid_svc)

{'C': [1, 10, 100, 1000], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4]}


In [48]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

model_forest = RandomForestClassifier()
model_nb = MultinomialNB()
model_svc = SVC()

In [49]:
#best parameters for RF(with BoW)
from sklearn.model_selection import RandomizedSearchCV
RF_RandomGrid = RandomizedSearchCV(estimator = model_forest, param_distributions = param_grid_forest, cv = 10, verbose=2, n_jobs = 4)
RF_RandomGrid.fit(x_train, y_train)
RF_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'n_estimators': 140,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 40}

In [42]:
#best parameters for MNB(with BoW)
NB_RandomGrid = RandomizedSearchCV(estimator = model_nb, param_distributions = param_grid_nb, cv = 10, verbose=2, n_jobs = 4)
NB_RandomGrid.fit(x_train, y_train)
NB_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'fit_prior': False, 'alpha': 1.0}

In [35]:
#best parameters for SVC(with BoW)
SVC_RandomGrid = RandomizedSearchCV(estimator = model_svc, param_distributions = param_grid_svc, cv = 10, verbose=2, n_jobs = 4)
SVC_RandomGrid.fit(x_train, y_train)
SVC_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'kernel': 'linear', 'degree': 1, 'C': 1}

In [50]:
#model establishment and results(BoW)
#Random Forest
model_forest = RandomForestClassifier(n_estimators=140,min_samples_split=10, min_samples_leaf=2, max_features='auto', max_depth=40)
model_forest.fit(x_train,y_train)
prediction = model_forest.predict(x_test)

from sklearn.metrics import classification_report 
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

          -1       0.95      0.25      0.40       144
           0       0.80      0.70      0.74       184
           1       0.64      0.94      0.76       272

    accuracy                           0.70       600
   macro avg       0.79      0.63      0.63       600
weighted avg       0.76      0.70      0.67       600



In [51]:
model_nb = MultinomialNB(alpha=1.0, fit_prior=False)
model_nb = model_nb.fit(x_train,y_train)
prediction_nb = model_nb.predict(x_test)

print(classification_report(y_test, prediction_nb))

              precision    recall  f1-score   support

          -1       0.52      0.47      0.50       144
           0       0.76      0.53      0.62       184
           1       0.62      0.78      0.69       272

    accuracy                           0.63       600
   macro avg       0.63      0.59      0.60       600
weighted avg       0.64      0.63      0.62       600



In [52]:
model_svc = SVC(kernel='linear',degree=1, C=1)
model_svc = model_svc.fit(x_train,y_train)
prediction_svc = model_svc.predict(x_test)

print(classification_report(y_test, prediction_svc))

              precision    recall  f1-score   support

          -1       0.60      0.59      0.59       144
           0       0.70      0.79      0.74       184
           1       0.79      0.73      0.76       272

    accuracy                           0.71       600
   macro avg       0.70      0.70      0.70       600
weighted avg       0.72      0.71      0.71       600



In [53]:
#TF-IDF
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(df_tfidf, df['senti_textblob'],test_size = 0.2)

In [54]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

model_forest = RandomForestClassifier()
model_nb = MultinomialNB()
model_svc = SVC()

In [55]:
#best parameters for RF(with TFIDF)
from sklearn.model_selection import RandomizedSearchCV
RF_RandomGrid = RandomizedSearchCV(estimator = model_forest, param_distributions = param_grid_forest, cv = 10, verbose=2, n_jobs = 4)
RF_RandomGrid.fit(x_train_2, y_train_2)
RF_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'n_estimators': 120,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 40}

In [56]:
NB_RandomGrid = RandomizedSearchCV(estimator = model_nb, param_distributions = param_grid_nb, cv = 10, verbose=2, n_jobs = 4)
NB_RandomGrid.fit(x_train_2, y_train_2)
NB_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'fit_prior': False, 'alpha': 0.5}

In [57]:
SVC_RandomGrid = RandomizedSearchCV(estimator = model_svc, param_distributions = param_grid_svc, cv = 10, verbose=2, n_jobs = 4)
SVC_RandomGrid.fit(x_train_2, y_train_2)
SVC_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'kernel': 'linear', 'degree': 3, 'C': 10}

In [60]:
#model establishment and results(TF-IDF)
#Random Forest
model_forest = RandomForestClassifier(n_estimators=120,min_samples_split=5, min_samples_leaf=2, max_features='auto', max_depth=40)
#model_forest = RandomForestClassifier()
model_forest.fit(x_train_2,y_train_2)
prediction = model_forest.predict(x_test_2)

from sklearn.metrics import classification_report 
print(classification_report(y_test_2, prediction))

              precision    recall  f1-score   support

          -1       0.87      0.19      0.31       145
           0       0.73      0.63      0.68       178
           1       0.61      0.92      0.73       277

    accuracy                           0.66       600
   macro avg       0.74      0.58      0.57       600
weighted avg       0.71      0.66      0.62       600



In [62]:
model_nb = MultinomialNB(alpha=0.5, fit_prior=False)
model_nb = model_nb.fit(x_train_2,y_train_2)
prediction_nb = model_nb.predict(x_test_2)

print(classification_report(y_test_2, prediction_nb))

              precision    recall  f1-score   support

          -1       0.50      0.48      0.49       145
           0       0.70      0.49      0.58       178
           1       0.64      0.77      0.70       277

    accuracy                           0.62       600
   macro avg       0.61      0.58      0.59       600
weighted avg       0.62      0.62      0.61       600



In [63]:
model_svc = SVC(C=10, kernel='linear',degree=3)
model_svc = model_svc.fit(x_train_2,y_train_2)
prediction_svc = model_svc.predict(x_test_2)

print(classification_report(y_test_2, prediction_svc))

              precision    recall  f1-score   support

          -1       0.58      0.60      0.59       145
           0       0.72      0.78      0.75       178
           1       0.77      0.71      0.74       277

    accuracy                           0.71       600
   macro avg       0.69      0.70      0.69       600
weighted avg       0.71      0.71      0.71       600



In [137]:
#Word2vec
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(tweet_vec_df, df['senti_textblob'],test_size = 0.2)

In [139]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

model_forest = RandomForestClassifier()
model_nb = MultinomialNB()
model_svc = SVC()

In [140]:
#best parameters for RF(with Word2vec)
from sklearn.model_selection import RandomizedSearchCV
RF_RandomGrid = RandomizedSearchCV(estimator = model_forest, param_distributions = param_grid_forest, cv = 10, verbose=2, n_jobs = 4)
RF_RandomGrid.fit(x_train_3, y_train_3)
RF_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'n_estimators': 160,
 'min_samples_split': 15,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 30}

In [141]:
SVC_RandomGrid = RandomizedSearchCV(estimator = model_svc, param_distributions = param_grid_svc, cv = 10, verbose=2, n_jobs = 4)
SVC_RandomGrid.fit(x_train_3, y_train_3)
SVC_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'kernel': 'linear', 'degree': 2, 'C': 1000}

In [142]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #handle negative
x_train_3a = scaler.fit_transform(x_train_3)
x_test_3a = scaler.fit_transform(x_test_3)
NB_RandomGrid = RandomizedSearchCV(estimator = model_nb, param_distributions = param_grid_nb, cv = 10, verbose=2, n_jobs = 4)
NB_RandomGrid.fit(x_train_3a, y_train_3)
NB_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'fit_prior': True, 'alpha': 10.0}

In [143]:
#model establishment and results(Word2Vec)
#Random Forest
model_forest = RandomForestClassifier(n_estimators=160,min_samples_split=15, min_samples_leaf=2, max_features='log2', max_depth=30)
model_forest.fit(x_train_3,y_train_3)
prediction = model_forest.predict(x_test_3)

from sklearn.metrics import classification_report 
print(classification_report(y_test_3, prediction))

              precision    recall  f1-score   support

          -1       0.47      0.12      0.19       152
           0       0.61      0.47      0.53       194
           1       0.51      0.82      0.63       254

    accuracy                           0.53       600
   macro avg       0.53      0.47      0.45       600
weighted avg       0.53      0.53      0.49       600



In [144]:
model_svc = SVC(C=1000, kernel='linear', degree=2)
model_svc = model_svc.fit(x_train_3,y_train_3)
prediction_svc = model_svc.predict(x_test_3)

print(classification_report(y_test_3, prediction_svc))

              precision    recall  f1-score   support

          -1       0.75      0.06      0.11       152
           0       0.71      0.50      0.59       194
           1       0.52      0.92      0.66       254

    accuracy                           0.56       600
   macro avg       0.66      0.49      0.45       600
weighted avg       0.64      0.56      0.50       600



In [146]:
from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# x_train_3 = scaler.fit_transform(x_train_3)
# x_test_3 = scaler.fit_transform(x_test_3)

model_nb = MultinomialNB(alpha=10, fit_prior=False)
model_nb = model_nb.fit(x_train_3a,y_train_3)
prediction_nb = model_nb.predict(x_test_3a)

print(classification_report(y_test_3, prediction_nb))

              precision    recall  f1-score   support

          -1       0.27      0.27      0.27       152
           0       0.56      0.28      0.38       194
           1       0.46      0.63      0.53       254

    accuracy                           0.43       600
   macro avg       0.43      0.40      0.39       600
weighted avg       0.44      0.43      0.42       600

